In [1]:
# importing packages
import pandas as pd
import numpy as np
import os
import platform
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels as sm
pd.options.mode.chained_assignment = None#preventing pointless error message

In [2]:
#loading dataset데이터셋 불러오기(데이터 링크는 삭제해 놓은 상태)
df=pd.read_csv("원본데이터경로")

In [3]:
#removing bad case
df=df[df.Participant != 1].reset_index()

In [4]:
#demographic info
dfDemo=df[["Participant", "Glass", "Gender", "Birth"]]
dfDemo.Glass.replace([1, 2, 3], ["Glasses", "No Glasses", "Contacts"], inplace=True)
dfDemo.Glass.value_counts()/4

No Glasses    13.0
Contacts       9.0
Glasses        3.0
Name: Glass, dtype: float64

In [5]:
#demographic info 2
dfDemo.Gender.value_counts()/4

M    17.0
F     8.0
Name: Gender, dtype: float64

In [6]:
#demographic info 3
dfDemo["Age"]=2021-dfDemo.Birth#calculating age from birth info
dfDemo.Age.describe().round(2)

count    100.00
mean      22.16
std        2.23
min       19.00
25%       20.00
50%       22.00
75%       24.00
max       26.00
Name: Age, dtype: float64

In [7]:
#label change 변인명 정리
OldLabel=["Treatment", "Q1",  "Q2",  "Q3",  "Q4",  "Q5",  "Q6",  "Q7",  "Q8",  "Q9",  "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26"]
NewLabel=["HMD", "SInv1",  "SInv2",  "SInv3",  "SInv4",  "SInv5",  "TInv1",  "TInv2",  "TInv3",  "TInv4",  "DynImm1", "DynImm2", "DynImm3", "DynImm4", "DynImm5", "RealImm1", "RealImm2", "RealImm3", "RealImm4", "Flow1", "Flow2", "Flow3", "Flow4", "Flow5", "Att1", "Att2", "Att3"]
df.rename(columns={i:j for i,j in zip(OldLabel, NewLabel)}, inplace=True)
df["HMD"]=df["HMD"].replace([0,1], ["HMD","Monitor"])

In [8]:
#composite variables 각 아이템을 변인으로 합침
df["SInv"]=(df['SInv1']+df['SInv2']+df['SInv3']+df['SInv4']+df['SInv5'])/5
df["TInv"]=(df['TInv1']+df['TInv2']+df['TInv3']+df['TInv4'])/4
df["DynImm"]=(df['DynImm1']+df['DynImm2']+df['DynImm3']+df['DynImm4']+df['DynImm5'])/5
df["RealImm"]=(df['RealImm1']+df['RealImm2']+df['RealImm3']+df['RealImm4'])/4
df["Flow"]=(df['Flow1']+df['Flow2']+df['Flow3']+df['Flow4']+df['Flow5'])/5
df["Att"]=(df['Att1']+df['Att2']+df['Att3'])/3

In [9]:
# 데이터 정리 준비
tasknum=["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24"]#이벤트 변호
objlist=["Obj"+x for x in tasknum]#도현변인명 리스트
rtlist=["RT"+x for x in tasknum]#반응속도변인명 리스트
rtmslist=["RTms"+x for x in tasknum]#반응속도변인명(밀리초) 리스트
alist=["A"+x for x in tasknum]#a stands for elements that participants should answer A변인은 답을 해야하는지(1) 말아야하는지(0)
clist=["C"+x for x in tasknum]#c stands for correct answer C변인=정답(0) 혹은 오답(1)
identifiers=["Participant", "Session", "HMD"]#각 이벤트 구분자
composites=["SInv", "TInv", "DynImm", "RealImm", "Flow", "Att"]#윗 셀에서 합친 변인
for i, j in zip (tasknum, rtlist):
    df["RTms"+i]=df[j]*1000#초를 밀리초로 변환

In [10]:
#위의 변인명에 맞춰 데이터 합체 정리
taskdf=df[identifiers+composites+objlist+rtmslist+alist+clist]

In [11]:
#1차정리된 데이터
taskdf

,Participant,Session,HMD,SInv,TInv,DynImm,RealImm,Flow,Att,Obj1,...,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24
0,2,1,HMD,2.0,3.50,2.4,2.00,2.6,2.666667,301.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2,Monitor,2.0,2.75,1.8,3.25,2.4,3.666667,111.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
2,2,3,HMD,4.2,4.50,2.2,3.25,3.4,4.000000,101.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,2,4,Monitor,2.8,5.00,1.8,3.25,3.2,4.333333,211.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
4,3,1,Monitor,3.0,3.00,1.6,1.50,3.2,4.666667,201.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,25,4,HMD,3.2,3.75,1.8,1.75,3.2,3.000000,300.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
96,26,1,NaN,4.0,3.25,2.2,4.00,3.8,3.000000,310.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
97,26,2,NaN,1.0,3.00,1.0,1.00,3.4,3.000000,311.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
98,26,3,NaN,4.0,3.75,2.0,4.00,3.8,4.000000,100.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [12]:
#filling NaN in HMD 26번 참여자에 HMD변인 공란 채우기
taskdf["HMD"][(taskdf.Participant==26) & (taskdf.Session==1)]="HMD"
taskdf["HMD"][(taskdf.Participant==26) & (taskdf.Session==2)]="Monitor"
taskdf["HMD"][(taskdf.Participant==26) & (taskdf.Session==3)]="HMD"
taskdf["HMD"][(taskdf.Participant==26) & (taskdf.Session==4)]="Monitor"

In [13]:
#위의 1차정리 데이터를 long데이터로 변환
dfl=pd.wide_to_long(
    taskdf,
    stubnames=["Obj", "RTms", "A", "C"],
    i=identifiers,
    j="Order",
).reset_index()
dfl

,Participant,Session,HMD,Order,TInv,DynImm,Att,Flow,RealImm,SInv,Obj,RTms,A,C
0,2,1,HMD,1,3.5,2.4,2.666667,2.6,2.0,2.0,301.0,5980.0,0.0,1.0
1,2,1,HMD,2,3.5,2.4,2.666667,2.6,2.0,2.0,101.0,5980.0,0.0,1.0
2,2,1,HMD,3,3.5,2.4,2.666667,2.6,2.0,2.0,100.0,5980.0,0.0,1.0
3,2,1,HMD,4,3.5,2.4,2.666667,2.6,2.0,2.0,210.0,6410.0,0.0,1.0
4,2,1,HMD,5,3.5,2.4,2.666667,2.6,2.0,2.0,211.0,6410.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,26,4,Monitor,20,2.5,1.2,3.000000,3.4,1.0,1.0,211.0,2380.0,1.0,1.0
2396,26,4,Monitor,21,2.5,1.2,3.000000,3.4,1.0,1.0,311.0,5980.0,1.0,0.0
2397,26,4,Monitor,22,2.5,1.2,3.000000,3.4,1.0,1.0,310.0,1710.0,1.0,1.0
2398,26,4,Monitor,23,2.5,1.2,3.000000,3.4,1.0,1.0,201.0,6410.0,1.0,0.0


In [14]:
#get shape 변인 Obj는 도형의 형태, 색상, 채도 정보를 코딩한 상태, 이를 각각 다른 변인으로 독립
def func(x):
    if x < 199:
        return 'circle'
    elif 199 < x < 299:
        return 'triagle' 
    else:
        return 'rectangle'
dfl['Shape']=dfl.Obj.apply(func)#Shape 도형모양 변인 생성

In [15]:
#parsing color and brightness 색상(Color)과 채도(Bright) 
digitone=dfl.Obj.dropna().astype(str).str[2].astype(int) #get a series of the first digits of non-nan numbers
digittwo=dfl.Obj.dropna().astype(str).str[1].astype(int) #get a series of the second digits of non-nan numbers
dfl["Bright"]=digitone#Obj 변인 첫째 자리=채도
dfl["Color"]=digittwo#Obj 변인 둘째 자리=색상
#recoding values
dfl["Bright"]=["vivid" if x==1 else "Dark" for x in dfl["Bright"]]#채도변인값을 숫자에서 자연어로 변환
dfl["Color"]=["blue" if x==1 else "Yellow" for x in dfl["Color"]]#색상변인값을 숫자에서 자연어로 변환

In [16]:
#fixing rt issue 반응시간이 6초 이상으로 잘못 기록된 부분 수정
dfl["RTms"]=[x if x<=5980 else 5980 for x in dfl["RTms"]]

In [17]:
cdfl=dfl[dfl["C"]==1]#df with only correct responses 정답만을 모은 데이터셋 분리
acdfl=cdfl[cdfl["A"]==1]#df that includes correct responses on second appearances TP데이터셋 분리
icdfl=dfl[dfl["C"]==0]#df with only INcorrect responses 오답만을 모은 데이터셋 분리

In [18]:
#정답률 (24번 이벤트 중 맞은 횟수) 변인 생성
correct=acdfl.groupby(["Participant", "Session"])["C"].sum().reset_index()
correct["Accu"]=correct.C/24

In [19]:
dfl.to_csv("dfl.csv")#정리된 데이터 저장

In [20]:
#정리된 이벤트 단위 데이터를 참여자 단위 데이터로 변환
dflTreatment=dfl.groupby(["Participant", "Session", "HMD"])["SInv", 'TInv', 'RealImm', 'DynImm', 'Flow', 'Att', 'RTms', 'C'].mean().reset_index()
dflTreatment["Accu"]=correct.Accu
dflTreatment

/var/folders/c4/369bq60n7ns52x9b25j96drr0000gn/T/ipykernel_72156/1534398669.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dflTreatment=dfl.groupby(["Participant", "Session", "HMD"])["SInv", 'TInv', 'RealImm', 'DynImm', 'Flow', 'Att', 'RTms', 'C'].mean().reset_index()


,Participant,Session,HMD,SInv,TInv,RealImm,DynImm,Flow,Att,RTms,C,Accu
0,2,1,HMD,2.0,3.50,2.00,2.4,2.6,2.666667,5633.333333,0.500000,0.041667
1,2,2,Monitor,2.0,2.75,3.25,1.8,2.4,3.666667,5265.833333,0.666667,0.208333
2,2,3,HMD,4.2,4.50,3.25,2.2,3.4,4.000000,4505.000000,1.000000,0.500000
3,2,4,Monitor,2.8,5.00,3.25,1.8,3.2,4.333333,4940.833333,0.708333,0.291667
4,3,1,Monitor,3.0,3.00,1.50,1.6,3.2,4.666667,3794.166667,0.750000,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...
95,25,4,HMD,3.2,3.75,1.75,1.8,3.2,3.000000,4156.666667,0.916667,0.458333
96,26,1,HMD,4.0,3.25,4.00,2.2,3.8,3.000000,4449.166667,0.916667,0.416667
97,26,2,Monitor,1.0,3.00,1.00,1.0,3.4,3.000000,4685.416667,0.791667,0.333333
98,26,3,HMD,4.0,3.75,4.00,2.0,3.8,4.000000,4244.583333,1.000000,0.500000


In [21]:
#정답/오답 형태(TP, TN, FP, FN)에 따른 반응속도 변인, 각 답 형태의 등장횟수에 대한 변인 생성
TPOS=dfl[(dfl["A"] == 1) & (dfl["C"] == 1)]
TNEG=dfl[(dfl["A"] == 0) & (dfl["C"] == 1)]
FPOS=dfl[(dfl["A"] == 0) & (dfl["C"] == 0)]
FNEG=dfl[(dfl["A"] == 1) & (dfl["C"] == 0)]
TPRT=TPOS.groupby(["Participant", "Session", "HMD"])["RTms"].mean().reset_index()
TNRT=TNEG.groupby(["Participant", "Session", "HMD"])["RTms"].mean().reset_index()
FPRT=FPOS.groupby(["Participant", "Session", "HMD"])["RTms"].mean().reset_index()
FNRT=FNEG.groupby(["Participant", "Session", "HMD"])["RTms"].mean().reset_index()
TPRT.rename(columns={"RTms":"TPRT"}, inplace=True)
TNRT.rename(columns={"RTms":"TNRT"}, inplace=True)
FPRT.rename(columns={"RTms":"FPRT"}, inplace=True)
FNRT.rename(columns={"RTms":"FNRT"}, inplace=True)
TPC=TPOS.groupby(["Participant", "Session", "HMD"])["C"].count().reset_index()
TNC=TNEG.groupby(["Participant", "Session", "HMD"])["C"].count().reset_index()
FPC=FPOS.groupby(["Participant", "Session", "HMD"])["C"].count().reset_index()
FNC=FNEG.groupby(["Participant", "Session", "HMD"])["C"].count().reset_index()
TPC.rename(columns={"C":"TPC"}, inplace=True)
TNC.rename(columns={"C":"TNC"}, inplace=True)
FPC.rename(columns={"C":"FPC"}, inplace=True)
FNC.rename(columns={"C":"FNC"}, inplace=True)
dflTreatment=dflTreatment.merge(TPRT, how="left", on=["Participant", "Session", "HMD"])
dflTreatment=dflTreatment.merge(TPC, how="left", on=["Participant", "Session", "HMD"])
dflTreatment=dflTreatment.merge(TNRT, how="left", on=["Participant", "Session", "HMD"])
dflTreatment=dflTreatment.merge(TNC, how="left", on=["Participant", "Session", "HMD"])
dflTreatment=dflTreatment.merge(FPRT, how="left", on=["Participant", "Session", "HMD"])
dflTreatment=dflTreatment.merge(FPC, how="left", on=["Participant", "Session", "HMD"])
dflTreatment=dflTreatment.merge(FNRT, how="left", on=["Participant", "Session", "HMD"])
dflTreatment=dflTreatment.merge(FNC, how="left", on=["Participant", "Session", "HMD"])

In [22]:
#데이터 확인
dflTreatment

,Participant,Session,HMD,SInv,TInv,RealImm,DynImm,Flow,Att,RTms,C,Accu,TPRT,TPC,TNRT,TNC,FPRT,FPC,FNRT,FNC
0,2,1,HMD,2.0,3.50,2.00,2.4,2.6,2.666667,5633.333333,0.500000,0.041667,1470.000000,1,5980.0,11,2170.000000,1.0,5980.0,11.0
1,2,2,Monitor,2.0,2.75,3.25,1.8,2.4,3.666667,5265.833333,0.666667,0.208333,3242.000000,5,5980.0,11,2530.000000,1.0,5980.0,7.0
2,2,3,HMD,4.2,4.50,3.25,2.2,3.4,4.000000,4505.000000,1.000000,0.500000,3030.000000,12,5980.0,12,NaN,NaN,NaN,NaN
3,2,4,Monitor,2.8,5.00,3.25,1.8,3.2,4.333333,4940.833333,0.708333,0.291667,3075.714286,7,5980.0,10,3675.000000,2.0,5980.0,5.0
4,3,1,Monitor,3.0,3.00,1.50,1.6,3.2,4.666667,3794.166667,0.750000,0.375000,1550.000000,9,5980.0,9,1783.333333,3.0,5980.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,25,4,HMD,3.2,3.75,1.75,1.8,3.2,3.000000,4156.666667,0.916667,0.458333,2201.818182,11,5980.0,11,3780.000000,1.0,5980.0,1.0
96,26,1,HMD,4.0,3.25,4.00,2.2,3.8,3.000000,4449.166667,0.916667,0.416667,2306.000000,10,5980.0,12,NaN,NaN,5980.0,2.0
97,26,2,Monitor,1.0,3.00,1.00,1.0,3.4,3.000000,4685.416667,0.791667,0.333333,2450.000000,8,5980.0,11,3150.000000,1.0,5980.0,4.0
98,26,3,HMD,4.0,3.75,4.00,2.0,3.8,4.000000,4244.583333,1.000000,0.500000,2509.166667,12,5980.0,12,NaN,NaN,NaN,NaN


In [23]:
#데이터 저장
dflTreatment.to_csv("dflTreatment.csv")

In [24]:
#통계 분석을 위해 R 구동, 이하는 R언어
%load_ext rpy2.ipython

In [ ]:
%%R
#R패키지 및 파일로드
#Load Packages 
OS<-.Platform$OS.type
if(OS=="unix"){
  Packages<-"/Users/stephensong/Documents/GitHub/R-repository/Default_Packages_to_Load.R"
}else if(OS=="windows"){
  Packages<-"C:/Users/steph/Onedrive/Documents/GitHub/R repository/Default_Packages_to_Load.R"
}else{
  print("ERROR: Packages NOT LOADED!!")
}
source(Packages)
#OS settings
OS<-.Platform$OS.type
if(OS=="unix"){
  workdirectory<-"/Users/stephensong/Documents/GitHub/2021KUprojects/VRnBack(UP)/01_Original Data Analysis"# MAC file path
}else if(OS=="windows"){
  workdirectory<-"C:/Users/steph/Onedrive/Documents/GitHub/VRecommerce" # windows file path
}else{
  print("ERROR: OS could not be identified")
}
#setwd
setwd(workdirectory)
getwd()
dfall<-read.csv("dfl.csv")
dfT<-read.csv("dflTreatment.csv")

In [26]:
%%R
#Per Individual 개인단위에서 TP의 반응속도가 HMD혹은 Monitor에 따라 차이가 나는지 t-test(one-tailed, paired) 
#paired t-tests
t1<-t.test(TPRT ~ HMD, data = dfT, paired = TRUE, alternative = "less")
t1


	Paired t-test

data:  TPRT by HMD
t = -1.7763, df = 49, p-value = 0.04095
alternative hypothesis: true difference in means is less than 0
95 percent confidence interval:
      -Inf -9.863229
sample estimates:
mean of the differences 
              -175.7066 



In [27]:
%%R
# 조건별 M과 SD확인
# Summarize data by groups
tvalue <- t1$statistic %>% formatC(digits = 2, format = "f")
pvalue <- t1$p.value %>% formatC(digits = 2, format = "f")
df <- t1$parameter
md <- dfT %>%  
  group_by(HMD) %>% 
  dplyr::summarize(N = length(TPRT), 
            Mean = mean(TPRT),
            SD = sd(TPRT)) 
# Summarize whole data set
td <- dfT %>% 
  dplyr::summarize(HMD = "Total",
                      N = length(TPRT), 
                      Mean = mean(TPRT),
                      SD = sd(TPRT))
# Combine data into one object
dd <- rbind(md,td)
t1
dd

# A tibble: 3 × 4
  HMD         N  Mean    SD
  <chr>   <int> <dbl> <dbl>
1 HMD        50 2275.  549.
2 Monitor    50 2451.  585.
3 Total     100 2363.  571.


In [28]:
%%R
#이벤트 단위 분석: 4가지 독립변인 동시적용 Repeated ANOVA
#Per Incidence
#ANOVA, All is NS, except for Shape * Bright
aov <- ezANOVA(
    data = dfall, 
    dv = RTms, 
    wid = Participant, 
    within = c(HMD, Shape, Bright, Color), 
    type = 3) # the last two arguments are optional
aov

R[write to console]: 경고:
R[write to console]:  Converting "Participant" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Converting "HMD" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Converting "Shape" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Converting "Bright" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Converting "Color" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Collapsing data to cell means. *IF* the requested effects are a subset of the full design, you must use the "within_full" argument, else results may be inaccurate.



$ANOVA
                   Effect DFn DFd          F          p p<.05          ges
2                     HMD   1  24 0.90365353 0.35127956       0.0014479421
3                   Shape   2  48 0.63533771 0.53415256       0.0022198537
4                  Bright   1  24 0.80355204 0.37893732       0.0012147348
5                   Color   1  24 0.59639040 0.44749714       0.0009009892
6               HMD:Shape   2  48 0.19432226 0.82403705       0.0006538977
7              HMD:Bright   1  24 1.04313403 0.31727921       0.0010161164
8            Shape:Bright   2  48 5.72954074 0.00586885     * 0.0110217621
9               HMD:Color   1  24 0.29998209 0.58895172       0.0002263344
10            Shape:Color   2  48 0.51457764 0.60101296       0.0014968728
11           Bright:Color   1  24 0.19331208 0.66410795       0.0003739366
12       HMD:Shape:Bright   2  48 1.94687361 0.15382571       0.0048748234
13        HMD:Shape:Color   2  48 0.32826608 0.72177533       0.0007415881
14       HMD:Brigh

In [29]:
%%R
#모양과 채도만으로 다시 ANOVA
#Focusing on Shape * Bright
aov2 <- ezANOVA(
    data = dfall, 
    dv = RTms, 
    wid = Participant, 
    within = c(Shape, Bright), 
    type = 3) # the last two arguments are optional
aov2

R[write to console]: 경고:
R[write to console]:  Converting "Participant" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Converting "Shape" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Converting "Bright" to factor for ANOVA.

R[write to console]: 경고:
R[write to console]:  Collapsing data to cell means. *IF* the requested effects are a subset of the full design, you must use the "within_full" argument, else results may be inaccurate.



$ANOVA
        Effect DFn DFd         F           p p<.05         ges
2        Shape   2  48 0.6359319 0.533843443       0.005500535
3       Bright   1  24 0.8436556 0.367492751       0.003118609
4 Shape:Bright   2  48 5.3822295 0.007781008     * 0.025392424

$`Mauchly's Test for Sphericity`
        Effect         W         p p<.05
2        Shape 0.8851844 0.2459733      
4 Shape:Bright 0.8902197 0.2625537      

$`Sphericity Corrections`
        Effect       GGe      p[GG] p[GG]<.05       HFe       p[HF] p[HF]<.05
2        Shape 0.8970093 0.51785059           0.9648406 0.528568420          
4 Shape:Bright 0.9010793 0.01009104         * 0.9697782 0.008423249         *



In [30]:
%%R
#평균값 표준편차값
#M & SD
dfall %>%
  group_by(Shape, Bright) %>%
  get_summary_stats(RTms, type = "mean_sd") %>%
  print(n = 100)#force print all

# A tibble: 6 × 6
  Shape     Bright variable     n  mean    sd
  <chr>     <chr>  <chr>    <dbl> <dbl> <dbl>
1 circle    Dark   RTms       393 4302. 1947.
2 circle    vivid  RTms       397 4044. 1960.
3 rectangle Dark   RTms       397 4325. 1860.
4 rectangle vivid  RTms       407 4225. 1886.
5 triagle   Dark   RTms       400 4123. 1959.
6 triagle   vivid  RTms       406 4313. 1906.
